https://towardsdatascience.com/keyword-extraction-with-bert-724efca412ea

In [19]:
doc = """
         I cook on a daily basis. Not only do I enjoy eating but I am also fascinated with the process of finding absolute favourite recipes through experiments. When I cook, I am often reminded of my ex-housemates who conducted their own experiments and also joined mine. Ironically, dining in our places provided us with more chances to try exquisite and rare food than when dining out. This is because we inspired each other by cooking together and it created synergies. 

Unfortunately, I live alone at the moment and I do not have such like-minded colleagues. Therefore, I invite my friends and acquaintances once in a while in order to obtain their feedback on my own cuisine. I am delighted when my guests dig in to my dishes without pause as if they are extremely ravenous. Overall, they rate the meals I prepared for them as flavourful. However, I often find it tricky to satisfy children as many of them are fussy eaters. 

I am conscious of what I eat. So when I cook, I avoid artificial ingredients and try to serve a nutritious and balanced diet for myself. Nowadays, many people consume highly processed foods and fizzy drinks. I am often frustrated that I have to gently decline when my friends ask me to go for junk food.
      """

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

n_gram_range=(1,1)
stop_words = 'english'

count = CountVectorizer(ngram_range=n_gram_range, stop_words = stop_words).fit([doc])
candidates = count.get_feature_names_out()

In [25]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

In [26]:
from sklearn.metrics.pairwise import cosine_similarity

top_n=5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

In [27]:
keywords

['cook', 'meals', 'housemates', 'cooking', 'recipes']

In [28]:
import numpy as np
import itertools

def max_sum_sim(doc_embedding, word_embeddings, words, top_n, nr_candidates):
    # Calculate distances and extract keywords
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # Get top_n words as candidates based on cosine similarity
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # Calculate the combination of words that are the least similar to each other
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [29]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['dishes', 'eating', 'diet', 'housemates', 'cooking']

In [30]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=20)

['daily', 'friends', 'dishes', 'eating', 'housemates']

In [31]:
import numpy as np

def mmr(doc_embedding, word_embeddings, words, top_n, diversity):

    # Extract similarity within words, and between words and the document
    word_doc_similarity = cosine_similarity(word_embeddings, doc_embedding)
    word_similarity = cosine_similarity(word_embeddings)

    # Initialize candidates and already choose best keyword/keyphras
    keywords_idx = [np.argmax(word_doc_similarity)]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    for _ in range(top_n - 1):
        # Extract similarities within candidates and
        # between candidates and selected keywords/phrases
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # Calculate MMR
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # Update keywords & candidates
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [32]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

['recipes', 'housemates', 'cooking', 'diet', 'meals']

In [33]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)

['recipes', 'frustrated', 'housemates', 'daily', 'live']